predict - inner buffer - mosaic - filter

In [5]:
#@title Import

import os
import sys

# change system patch to be able to access the custom modules (functions, normalization)
# src: https://colab.research.google.com/drive/1uvHuizCBqFgvbCwEhK7FvU8JW0AfxgJw#scrollTo=psH0aLrvoh78
module_path = os.path.abspath(os.path.join('/home/lucie/GDA_UGS/ESA_GDA_UGS/GDA_urban'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import glob

import csv

from osgeo import osr, gdal
import json

import keras
from keras import utils
from keras.applications import VGG16
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, Concatenate, BatchNormalization, Activation, Dropout
from keras import metrics
from keras.models import Model
from keras.callbacks import EarlyStopping

import tensorflow as tf

#from PIL import Image
from matplotlib import pyplot as plt

#import functions
import functions_predict
import normalization

from pathlib import Path

import subprocess

#from sklearn.metrics import classification_report, confusion_matrix

In [11]:
#@title Parameters
#size = 224
size = 448

class_codes = list(range(1,9)) #NBS Kinshasa
#class_codes = list(range(0,8)) #Amman P2

base_dir=Path('/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa')
#base_dir=Path('/home/lucie/GDA_UGS/ESA_GDA_UGS/GDA_urban/Phase2')

#version='scenario_01'
version='scenario_06'
subversion='1'
model_filepath = base_dir.joinpath('05_models',f'model_{version}_{subversion}',f'model_{version}_{subversion}.json')

model_weights = base_dir.joinpath('05_models',f'model_{version}_{subversion}',f'model_{version}_{subversion}.h5')

work_dir = base_dir.joinpath('02_model_input','data_predict_outsideAOI_3locations')
#work_dir = base_dir.joinpath('testing',f'version_{version}_{subversion}','predict_outsideAOI_in_v4')

model_output_parent_folder = base_dir.joinpath('04_prediction',f'model_{version}_{subversion}')
#model_output_parent_folder = base_dir.joinpath('testing',f'version_{version}_{subversion}','predict_outsideAOI_out_v4')

band_list= ['b01', 'b02', 'b03']


In [12]:
def read_file_names(dir_img):

  #file_names=np.array(glob.glob(dir_img+"/*.*"))
  file_names=np.array(glob.glob(str(dir_img.joinpath('*.*'))))

  from pathlib import Path
  fileid = [Path(x).name[0:3] for x in file_names]
  file_names = [x for _,x in sorted(zip(fileid,file_names))]

  return file_names

def output_export(raster_list,L_test,model_output,size,class_names,output_folder,version,export=False):
  
  temp_list=[]

  if export==False:
    for i in range(L_test):
      temp = model_output[(i,)]

      predict_class = probab2class(temp, size, class_names)
      array = predict_class
      
      temp_list.append(array)

    out2 = np.stack(temp_list, axis=0)

  else:
    if not output_folder.exists():
        os.makedirs(output_folder)

    output_path = f'{output_folder}/'

    for i in range(L_test):
      temp = model_output[(i,)]

      predict_class = functions_predict.probab2class(temp, size, class_names)
      array = predict_class
      temp_list.append(array)

      rasterfn = raster_list[i]
      name = os.path.basename(os.path.normpath(raster_list[i]))

      newRasterfn = output_folder.joinpath(f'{name}')

      functions_predict.array2raster(rasterfn, str(newRasterfn), array, data_type=gdal.GDT_Byte)
    out2 = np.stack(temp_list, axis=0)

  return out2


In [14]:
## DATA - set paths HERE

os.makedirs(model_output_parent_folder, exist_ok=True)

#list_cells=['36','38','9']
#list_cells=os.listdir('/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_test')
list_cells=os.listdir(work_dir)

for part in list_cells:
    print(part)

    subdir=work_dir.joinpath(part)
    os.walk(subdir)

    list_cells_subfolder=os.listdir(subdir)
    
    for subpart in list_cells_subfolder:
        
        if subpart.endswith('.tif'):
            continue
            
        else:
            input_data = subdir.joinpath(subpart)
            print(input_data)
  
        output_folder = model_output_parent_folder.joinpath(part)
        print(output_folder)   

        #input data preprocessing - load, normalize
        predict_data = read_file_names(input_data)
        print(predict_data)

        print('data normalization...')
        predict_data_normalized, _ = normalization.normalization(predict_data, band_list)

        #src:https://machinelearningmastery.com/save-load-keras-deep-learning-models/
        from keras.models import model_from_json

        # load json and create model
        print('model loading...')
        json_file = open(model_filepath, 'r')
        print(json_file)
        loaded_model_json = json_file.read()
        json_file.close()

        loaded_model = model_from_json(loaded_model_json)

        # load weights into new model
        loaded_model.load_weights(model_weights, 'r')
        print("Loaded model from disk")

        # predict 
        print('prediction...')
        prediction=loaded_model.predict(x=predict_data_normalized)

        #export
        print('export...')
        raster_list = predict_data
        L_pred=len(predict_data)

        model_output_classif=output_export(raster_list,L_pred,prediction,size,class_codes,output_folder,version,export=True)
        

        print('')


img_tile_310
/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_310/img
/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_310
['/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_310/img/0001_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_310/img/0002_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_310/img/0003_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_310/img/0004_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_310/img/0005_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_310/img/0006_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_m

model loading...
<_io.TextIOWrapper name='/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/05_models/model_scenario_06_1/model_scenario_06_1.json' mode='r' encoding='UTF-8'>
Loaded model from disk
prediction...
2/2 [==============================] - 252s 88s/step
export...

img_tile_617
/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_617/img
/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/04_prediction/model_scenario_06_1/img_tile_617
['/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_617/img/0001_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_617/img/0002_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_617/img/0003_img.tif', '/home/lucie/GDA_UGS/ESA_GDA_UGS/Kinshasa/02_model_input/data_predict_outsideAOI_3locations/img_tile_617/img/0004_img.tif', '/home/lucie/GDA_UGS/ESA_GD